In [2]:
import numpy as np
import pandas as pd
from submodlib import FacilityLocationFunction, DisparitySumFunction, DisparityMinFunction, LogDeterminantFunction, FacilityLocationMutualInformationFunction
import os
import shutil
import numpy as np
import time

In [3]:
groundData = pd.read_excel('2_img_features_lake.xlsx')

In [4]:
repData  = pd.read_excel('3_img_features_lake.xlsx')
len(repData)

512

In [5]:
repData_transpose = repData.transpose()

In [6]:
groundData_transpose = groundData.transpose()

num_images = len(groundData_transpose)
num_features = 512
budget = 1000

# algo = 'LogDeterminantFunction'
algo = 'FacilityLocationMutualInformationFunction'
from submodlib import LogDeterminantMutualInformationFunction
etas = [1.8]
row = 0
index = 1

supported_algos = ['FacilityLocationFunction', 'DisparitySumFunction', 'DisparityMinFunction', 'LogDeterminantFunction']

In [ ]:
len(groundData_transpose)

In [8]:
# grounds = [list(groundData_transpose[x]) for x in groundData_transpose.columns]
# len(grounds)

In [ ]:
start_time = time.time()
print('Starting selection algo ...')
if algo == 'FacilityLocationFunction':
    # objFL = FacilityLocationFunction(n=len(grounds), data=groundData, separate_rep=True, n_rep=36, data_rep=repData, mode="dense", metric="euclidean")
    objFL = FacilityLocationFunction(n=num_images, data=np.array(groundData_transpose), separate_rep=False, mode="dense", metric="euclidean")
    greedyList = objFL.maximize(budget=budget ,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
elif algo == 'DisparitySumFunction':
    objFL = DisparitySumFunction(n=num_images, data=np.array(groundData_transpose), mode="dense", metric="euclidean")
    greedyList = objFL.maximize(budget=budget ,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
elif algo == 'DisparityMinFunction':
    objFL = DisparityMinFunction(n=num_images, data=np.array(groundData_transpose), mode="dense", metric="euclidean")
    greedyList = objFL.maximize(budget=budget ,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
elif algo == 'LogDeterminantFunction':
    lambda_value = 1
    objFL = LogDeterminantFunction(n=num_images, data=np.array(groundData_transpose), mode="dense", metric="euclidean", lambdaVal=lambda_value)
    greedyList = objFL.maximize(budget=budget ,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
elif algo == 'FacilityLocationMutualInformationFunction':
    obj = FacilityLocationMutualInformationFunction(n=num_images, num_queries=len(repData_transpose), data=np.array(groundData_transpose),queryData=np.array(repData_transpose), metric="euclidean", magnificationEta=etas)
    greedyList = obj.maximize(budget=budget,optimizer='NaiveGreedy', stopIfZeroGain=False,
                              stopIfNegativeGain=False, verbose=False)

end_time = time.time()

print(f'Time taken - {np.round((end_time - start_time) / 60, 2)}')

# greedys = [[grounds[i][x[0]] for x in greedyList] for i in range(num_features)]

Starting selection algo ...


In [ ]:
greedyList

In [18]:
selected_image_idxs = [greedyList[i][0] for i in range(len(greedyList))]

In [19]:
selected_image_names = [groundData_transpose.index[i] for i in selected_image_idxs]

In [20]:
# Splitting data for creating train_baseline
try:
    os.mkdir('train_w_submodlib')
except FileExistsError:
    pass

try:
    os.mkdir('train_w_submodlib/normal')
except FileExistsError:
    pass

try:
    os.mkdir('train_w_submodlib/lung_op')
except FileExistsError:
    pass

In [21]:
for i in range(len(selected_image_names)):

    # print(selected_image_names[i])
    img_file_source = os.path.join('all_to_clean', selected_image_names[i])
    if selected_image_names[i].split('.')[0][0] == 'l':
        img_file_destination = os.path.join('train_w_submodlib', 'lung_op', selected_image_names[i])
    elif selected_image_names[i].split('.')[0][0] == 'n':
        img_file_destination = os.path.join('train_w_submodlib', 'normal', selected_image_names[i])
    else:
        print('Image file destination not defined')
        img_file_destination = None

    print(img_file_source)
    print(img_file_destination)
    shutil.copy(img_file_source, img_file_destination)

all_to_clean/l-generated_226.jpg
train_w_submodlib/lung_op/l-generated_226.jpg
all_to_clean/n-generated_410.jpg
train_w_submodlib/normal/n-generated_410.jpg
all_to_clean/l-generated_572.jpg
train_w_submodlib/lung_op/l-generated_572.jpg
all_to_clean/n-generated_1064.jpg
train_w_submodlib/normal/n-generated_1064.jpg
all_to_clean/n-generated_249.jpg
train_w_submodlib/normal/n-generated_249.jpg
all_to_clean/l-generated_814.jpg
train_w_submodlib/lung_op/l-generated_814.jpg
all_to_clean/n-generated_27.jpg
train_w_submodlib/normal/n-generated_27.jpg
all_to_clean/l-generated_285.jpg
train_w_submodlib/lung_op/l-generated_285.jpg
all_to_clean/l-generated_1391.jpg
train_w_submodlib/lung_op/l-generated_1391.jpg
all_to_clean/n-generated_1121.jpg
train_w_submodlib/normal/n-generated_1121.jpg
all_to_clean/n-generated_92.jpg
train_w_submodlib/normal/n-generated_92.jpg
all_to_clean/n-generated_451.jpg
train_w_submodlib/normal/n-generated_451.jpg
all_to_clean/n-generated_259.jpg
train_w_submodlib/normal

In [24]:
# Creating split for training WGAN for normal and pneumonia

# Splitting data for creating train_baseline

try:
    os.mkdir('gan_train')
except FileExistsError:
    pass

try:
    os.mkdir('gan_train/normal')
except FileExistsError:
    pass

try:
    os.mkdir('gan_train/lung_op')
except FileExistsError:
    pass

try:
    os.mkdir('gan_train/normal/normal')
except FileExistsError:
    pass

try:
    os.mkdir('gan_train/lung_op/lung_op')
except FileExistsError:
    pass

In [27]:
for i in range(len(selected_image_names)):

    # print(selected_image_names[i])

    img_file_source = os.path.join('lake', selected_image_names[i])

    if selected_image_names[i].split('.')[0][0] == 'L':
        img_file_destination = os.path.join('gan_train', 'lung_op', 'lung_op', selected_image_names[i])
    elif selected_image_names[i].split('.')[0][0] == 'N':
        img_file_destination = os.path.join('gan_train', 'normal', 'normal', selected_image_names[i])
    else:
        print('Image file destination not defined')
        img_file_destination = None

    # print(img_file_source)
    # print(img_file_destination)

    shutil.copy(img_file_source, img_file_destination)